# 1. Import libraries

In [1]:
import io
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn import tree

In [3]:
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['figure.figsize'] = 10,5

In [4]:
def num_var(x,y,z,i,j):
    dtree = tree.DecisionTreeClassifier(random_state=42, max_depth=i, min_samples_leaf=j)
    dtree = dtree.fit(df[[x]],df[y])
    
    df[z] = dtree.apply(df[[x]])
    gb = df.groupby(z).agg({'account':'count', x:['min','max'], y:'mean'}).reset_index()
    display(gb)

In [5]:
def cat_var(x,y):
    gb = df.groupby(x).agg({'account':'count',y:'mean'}).reset_index()
    display(gb)

# 2. Import datasets

In [6]:
df = pd.read_csv("../outputs/master.csv")
df.shape

(1000, 9)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
account        1000 non-null int64
loan           1000 non-null object
loan_amount    1000 non-null int64
months         1000 non-null int64
history        1000 non-null object
age            1000 non-null int64
purpose        1000 non-null object
property       1000 non-null object
Y              1000 non-null int64
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


# 3. Numerical - 3

In [8]:
#num_var('loan_amount', 'Y', 'bin', 3, 50)
#num_var('months', 'Y', 'bin', 3, 50)
#num_var('age', 'Y', 'bin', 3, 50)

In [9]:
df['loan_amount'] = np.where(df['loan_amount']<=1372000, 0.32,
                             np.where(df['loan_amount']<=3446000, 0.24,
                                      np.where(df['loan_amount']<=3913000, 0.08,
                                               np.where(df['loan_amount']<=5003000, 0.48,
                                                        np.where(df['loan_amount']<=7824000, 0.32, 0.54)))))

df['months'] = np.where(df['months']<=8, 0.10,
                        np.where(df['months']<=11, 0.20,
                                 np.where(df['months']<=15, 0.24,
                                          np.where(df['months']<=33, 0.32,
                                                   np.where(df['months']<=42, 0.42, 0.56)))))

df['age'] = np.where(df['age']<=23, 0.39,
                     np.where(df['age']<=25, 0.45,
                              np.where(df['age']<=27, 0.27,
                                       np.where(df['age']<=34, 0.33,
                                                np.where(df['age']<=36, 0.15, 0.27)))))


In [10]:
cat_var('loan_amount','Y')
cat_var('months','Y')
cat_var('age','Y')

,loan_amount,account,Y
0,0.08,60,0.083333
1,0.24,427,0.243560
2,0.32,365,0.317808
3,0.48,73,0.465753
4,0.54,75,0.546667


,months,account,Y
0,0.10,94,0.106383
1,0.20,86,0.197674
2,0.24,251,0.247012
3,0.32,399,0.323308
4,0.42,100,0.420000
5,0.56,70,0.571429


,age,account,Y
0,0.15,79,0.151899
1,0.27,474,0.259494
2,0.33,257,0.330739
3,0.39,105,0.400000
4,0.45,85,0.447059


# 4. Categorical - 3

In [11]:
#cat_var('history','Y')
#cat_var('purpose','Y')
#cat_var('property','Y')

In [12]:
df['history'] = np.where(df['history']=='all loans at this bank paid back duly', 0.57,
                         np.where(df['history']=='critical/pending loans at other banks', 0.18,
                                  np.where(df['history']=='no loans taken/all loans paid back duly', 0.63, 0.33)))

df['purpose'] = np.where(df['purpose']=='career development', 0.12, 
                         np.where(df['purpose']=='used vehicle', 0.18,
                                  np.where(df['purpose']=='electronic equipment', 0.21,
                                           np.where(df['purpose'].isin(['FF&E','domestic appliances','business','repair costs','new vehicle']), 0.36,0.42))))

df['property'] = np.where(df['property']=='real estate', 0.20,
                          np.where(df['property']=='MISSING', 0.45, 0.30))

In [13]:
cat_var('history','Y')
cat_var('purpose','Y')
cat_var('property','Y')

,history,account,Y
0,0.18,293,0.170648
1,0.33,618,0.318770
2,0.57,49,0.571429
3,0.63,40,0.625000


,purpose,account,Y
0,0.12,9,0.111111
1,0.18,103,0.165049
2,0.21,280,0.221429
3,0.36,546,0.353480
4,0.42,62,0.435484


,property,account,Y
0,0.20,282,0.212766
1,0.30,564,0.306738
2,0.45,154,0.435065


# 5. Export Dataset

In [14]:
df = df[['loan_amount', 'months', 'history', 'age', 'purpose', 'property', 'Y']]
df.to_csv('../outputs/model.csv', index=False)
df.shape

(1000, 7)